In [48]:
from data_loader import PykrxDataLoader
from backtesting import Backtest, Strategy
import pandas as pd

In [49]:
pykrxdata = PykrxDataLoader('20220101', '20250331')

In [ ]:
df = pykrxdata.load_stock_data(['005930'], 'd')

In [58]:
df.rename(columns={'open' : 'Open', 'high' : 'High', 'low' : 'Low', 'close' : 'Close', 'volume' : 'Volume'} , inplace = True)

In [59]:
df

,Open,High,Low,Close,Volume
date,,,,,
2022-01-03,79400,79800,78200,78600,13502112
2022-01-04,78800,79200,78300,78700,12427416
2022-01-05,78800,79000,76400,77400,25470640
2022-01-06,76700,77600,76600,76900,12931954
2022-01-07,78100,78400,77400,78300,15163757
...,...,...,...,...,...
2025-03-25,60900,61100,59500,59800,17259455
2025-03-26,59800,61400,59700,61400,16431645
2025-03-27,60900,62000,60800,61800,20389790


In [60]:
class HighLowBreakoutStrategy(Strategy):
    # 전략 파라미터
    n_high_period = 20  # 신고가 기간
    n_low_period = 10   # 신저가 기간

    def init(self):
        # 가격 데이터를 Series로 변환하여 rolling 계산 준비
        close_prices = self.data.Close 
        
        # 20일 신고가 계산 (당일 종가 포함)
        self.rolling_market_high = self.I(
            lambda x, n: pd.Series(x).rolling(n, min_periods=n).max(),
            close_prices,
            self.n_high_period
        )
        
        # 10일 신저가 계산 (당일 종가 포함)
        self.rolling_market_low = self.I(
            lambda x, n: pd.Series(x).rolling(n, min_periods=n).min(),
            close_prices,
            self.n_low_period
        )

    def next(self):
        # 현재 종가 (오늘 종가)
        current_close = self.data.Close[-1]

        # 매수 조건:
        # 1. 현재 보유 포지션이 없음
        # 2. 오늘 종가가 오늘을 포함한 과거 20일 중 최고가(신고가) 이상인 경우
        if not self.position and current_close >= self.rolling_market_high[-1]:
            # 이 매수 주문은 다음 날 시가에 체결됩니다.
            self.buy() 

        # 매도 조건:
        # 1. 현재 롱 포지션을 보유 중
        # 2. 오늘 종가가 오늘을 포함한 과거 10일 중 최저가(신저가) 이하인 경우
        elif self.position.is_long and current_close <= self.rolling_market_low[-1]:
            # 이 매도 주문은 다음 날 시가에 체결됩니다.
            self.position.close() 

In [61]:
bt = Backtest(
            df,
            HighLowBreakoutStrategy,
            cash=10_000_000,  # 초기 자본금 1천만원
            commission=.00015, # 수수료 0.015%
            exclusive_orders=True 
        )

In [63]:
stats = bt.run()
print("백테스팅 결과:")
print(stats)
        
# 백테스팅 상세 거래 내역
print("\n거래 내역:")
print(stats._trades)

# 결과 시각화 (웹 브라우저에 HTML 파일로 결과가 표시됩니다)
# plot_filename 인자를 설정하여 파일로 저장할 수도 있습니다. (예: bt.plot(filename='backtest_results.html'))
bt.plot()
print("\n백테스팅 결과 차트가 웹 브라우저에 표시됩니다.")
print("Jupyter Notebook 환경에서는 바로 차트가 나타날 수 있습니다.")

백테스팅 결과:
Start                     2022-01-03 00:00:00
End                       2025-03-31 00:00:00
Duration                   1183 days 00:00:00
Exposure Time [%]                    34.42623
Equity Final [$]                  9622259.035
Equity Peak [$]                  11799643.165
Commissions [$]                     50724.555
Return [%]                           -3.77741
Buy & Hold Return [%]               -21.14598
Return (Ann.) [%]                    -1.21619
Volatility (Ann.) [%]                12.92237
CAGR [%]                             -0.81689
Sharpe Ratio                         -0.09412
Sortino Ratio                          -0.133
Calmar Ratio                         -0.06591
Alpha [%]                             1.43954
Beta                                  0.24671
Max. Drawdown [%]                   -18.45297
Avg. Drawdown [%]                    -5.77801
Max. Drawdown Duration      402 days 00:00:00
Avg. Drawdown Duration       94 days 00:00:00
# Trades                 


백테스팅 결과 차트가 웹 브라우저에 표시됩니다.
Jupyter Notebook 환경에서는 바로 차트가 나타날 수 있습니다.
